# 🔬 Lab: Validação dos Retrievers Individuais

Este notebook demonstra **passo a passo** como funcionam os três tipos de retrieval implementados no projeto, com ênfase especial no **retriever de grafo** (entidades + embeddings).

## Objetivos
1. **TF-IDF Retriever**: Entender retrieval baseado em sobreposição lexical
2. **Dense Retriever**: Compreender embeddings semânticos (MiniLM vs BGE)
3. **Graph Retriever**: Explorar em profundidade a extração de entidades, IDF e agregação ponderada

Usaremos um mini-corpus de 5 documentos e 3 queries para validação.

## 📦 Setup: Imports e Configuração do Ambiente

Primeiro, garantimos que o Python encontra o repositório e importamos as dependências necessárias.

In [2]:
# Se estiver em um ambiente limpo, descomente conforme necessário:
# %pip install -U sentence-transformers transformers torch scikit-learn spacy scispacy

import sys
import warnings
from pathlib import Path
import numpy as np

repo_root = Path.cwd()
while repo_root != repo_root.parent and repo_root.name != "hybrid-retrieval":
    repo_root = repo_root.parent
repo_root_str = str(repo_root)
if repo_root_str not in sys.path:
    sys.path.insert(0, repo_root_str)

print("Repo root:", repo_root)

# Imports dos schemas do projeto
from src.datasets.schema import Document, Query

# Silencia warnings de modelos
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

Repo root: /teamspace/studios/this_studio/hybrid-retrieval


## 📚 Criação do Mini-Corpus

Exemplos:

Podemos criar 5 documentos com temas variados para testar os diferentes comportamentos dos retrievers:
- **D1**: COVID-19 e vacinas (domínio médico, entidades: "COVID-19", "mRNA vaccines")
- **D2**: Mercados financeiros (domínio financeiro, entidades: "equity market", "US economy")
- **D3**: Graph Neural Networks (domínio técnico, entidades: "GNNs", "entity classification")
- **D4**: Nutrição (domínio saúde, entidades: "apples", "fiber", "vitamins")
- **D5**: Large Language Models (domínio ML, entidades: "MiniLM", "BGE")

E 3 queries que exploram diferentes aspectos:
- **Q1**: Busca sobre eficácia de vacinas (termos relacionados a D1)
- **Q2**: Busca sobre impacto de taxas de juros (relacionada a D2)
- **Q3**: Busca sobre embeddings de entidades e grafos (relacionada a D3 e D5)

Segue uma expansão dessa ideia:

In [3]:
from src.datasets.schema import Document, Query

docs = [
    # — Ciência / Biomédico —
    Document(doc_id="B1", title="COVID-19 mRNA vaccines", text="Clinical trials show efficacy of mRNA vaccines against COVID-19."),
    Document(doc_id="B2", title="mRNA vaccine safety profile", text="Adverse effects and safety profile of mRNA COVID-19 vaccines."),
    Document(doc_id="B3", title="SARS-CoV-2 variants", text="Variants of concern and vaccine effectiveness in real-world evidence."),
    Document(doc_id="B4", title="Clinical trials methodology", text="Randomized controlled trials for vaccines and therapies."),
    Document(doc_id="B5", title="Long COVID studies", text="Long-term cardiovascular effects following COVID-19 infection."),

    # — Saúde / Nutrição (NFCorpus-like) —
    Document(doc_id="N1", title="Apples and dietary fiber", text="Apples provide soluble fiber and vitamins that reduce health risks."),
    Document(doc_id="N2", title="Mediterranean diet", text="Lower cardiovascular risk associated with Mediterranean diet."),
    Document(doc_id="N3", title="Vitamin D supplementation", text="Vitamin D reduces risk of bone fractures in elderly population."),
    Document(doc_id="N4", title="Fiber and gut microbiome", text="Dietary fiber improves gut microbiome and metabolic health."),
    Document(doc_id="N5", title="Sugar intake guidelines", text="High sugar intake increases obesity and diabetes risk."),

    # — Finanças (FIQA-like) —
    Document(doc_id="F1", title="Interest rates and equity volatility", text="Federal Reserve policy impacts equity market volatility and valuations."),
    Document(doc_id="F2", title="US economy outlook", text="US CPI and interest rates drive consumer spending and growth."),
    Document(doc_id="F3", title="NASDAQ and tech stocks", text="NASDAQ index rallies as AI revenues beat expectations."),
    Document(doc_id="F4", title="S&P 500 valuation", text="S&P 500 price-to-earnings expands as rates stabilize."),
    Document(doc_id="F5", title="Dow Jones and dividends", text="Dividend yields support Dow Jones performance amid rate cuts."),

    # — ML / IR / Grafos —
    Document(doc_id="M1", title="Entity embeddings for retrieval", text="BGE and MiniLM embeddings improve retrieval and reranking."),
    Document(doc_id="M2", title="Graph neural networks", text="GNNs for entity classification and link prediction in knowledge graphs."),
    Document(doc_id="M3", title="Hybrid retrieval (dense+lexical+graph)", text="Tri-modal fusion with TF-IDF, semantic and entity graph signals."),
    Document(doc_id="M4", title="RAG pipelines", text="Retrieval augmented generation with reranking and hallucination control."),
    Document(doc_id="M5", title="Vector databases", text="FAISS IVFPQ indexing for large-scale nearest neighbor search."),

    # — Pontes entre domínios (para DF > 1 em termos transversais) —
    Document(doc_id="X1", title="Cardiovascular risk and diet", text="Mediterranean diet and apples reduce cardiovascular risk."),
    Document(doc_id="X2", title="COVID-19 and US economy", text="COVID-19 shocks affect US labor market and consumer demand."),
    Document(doc_id="X3", title="AI revenues and S&P 500", text="AI-related revenues boost S&P 500 and NASDAQ tech valuations."),
    Document(doc_id="X4", title="Clinical trials and graph methods", text="Graph-based methods assist clinical trials entity linking."),
]

queries = [
    Query(query_id="Q1", text="Are mRNA vaccines effective against COVID-19?"),
    Query(query_id="Q2", text="How do interest rates impact equity volatility and valuations?"),
    Query(query_id="Q3", text="Entity embeddings and graph-based retrieval methods in knowledge graphs"),
    Query(query_id="Q4", text="Do apples and Mediterranean diet reduce cardiovascular risk?"),
    Query(query_id="Q5", text="Are AI-related revenues influencing NASDAQ and S&P 500?"),
]

print(f"✓ Corpus: {len(docs)} documentos")
print(f"✓ Queries: {len(queries)} consultas")

✓ Corpus: 24 documentos
✓ Queries: 5 consultas


---
# 🔤 Parte 1: TF-IDF Retriever (Sinal Lexical)

## O que é TF-IDF?
- **Term Frequency (TF)**: Frequência do termo no documento
- **Inverse Document Frequency (IDF)**: Penaliza termos comuns no corpus
- **Vetor TF-IDF**: Cada documento é representado como um vetor esparso de tamanho = vocabulário

## Como funciona?
1. **Fit**: Constrói vocabulário do corpus e calcula IDF para cada termo
2. **Transform**: Converte documento → vetor TF-IDF normalizado (L2)
3. **Busca**: Similaridade por inner-product (equivalente a cosseno após normalização)

## Vantagens
✓ Rápido e interpretável  
✓ Bom para matching exato de termos  

## Limitações
✗ Não captura sinônimos ou paráfrases  
✗ Sensível a variações morfológicas  

In [4]:
from src.retrievers.tfidf_faiss import TFIDFRetriever

tfidf_r = TFIDFRetriever(
    dim=2000,             # vocabulário maior
    min_df=2,             # filtra termos raros; DF>=2 para corpus ampliado
    backend="sklearn",
    use_faiss=True,
    artifact_dir=None,
    index_name="tfidf.index",
)

print("🔧 Construindo índice TF-IDF...")
tfidf_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
tfidf_results = tfidf_r.retrieve(queries, k=3)

for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = tfidf_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        title = next(d.title for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {title}")

🔧 Construindo índice TF-IDF...
2025-10-30 00:38:50 | INFO     | retriever.tfidf | [tfidf_faiss.py:66] | 🚀 Building TF-IDF Index (24 documentos)
2025-10-30 00:38:50 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Fit TF-IDF no corpus - iniciando...
2025-10-30 00:38:50 | INFO     | tfidf.vectorizer | [logging.py:199] | ⏱️  Fit TF-IDF - iniciando...


2025-10-30 00:38:52 | INFO     | tfidf.vectorizer | [logging.py:220] | ✓ Fit TF-IDF - concluído em 1.38s
2025-10-30 00:38:52 | INFO     | tfidf.vectorizer | [tfidf_vectorizer.py:19] | ✓ TF-IDF fitted: vocab_size=41
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [logging.py:220] | ✓ Fit TF-IDF no corpus - concluído em 1.39s
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Encoding documents (TF-IDF) - iniciando...
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [logging.py:220] | ✓ Encoding documents (TF-IDF) - concluído em 6.6ms
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [logging.py:199] | ⏱️  Construindo FAISS IndexFlatIP - iniciando...
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [logging.py:220] | ✓ Construindo FAISS IndexFlatIP - concluído em 9.7ms
2025-10-30 00:38:52 | INFO     | retriever.tfidf | [tfidf_faiss.py:85] |   ✓ FAISS IndexFlatIP: 24 vetores, dim=2000

🔍 Executando retrieval (top-3 por query)...

📊 Query [Q1]: Are mRNA vacci

### 📈 Análise dos Resultados TF-IDF

**Observe**:
- Queries com **termos exatos** do corpus (ex: "mRNA", "vaccines", "COVID") ranqueiam bem documentos correspondentes
- Queries com **paráfrases** (ex: "effective" vs "efficacy") podem ter scores mais baixos
- O TF-IDF é **lexical**: não entende que "interest rates" e "taxas de juros" são sinônimos

**Exemplo**: Q1 ranquea B1 no topo, pois compartilha "mRNA", "vaccines", "COVID".

---
# 🧠 Parte 2: Dense Retriever (Embeddings Semânticos)

## O que é Dense Retrieval?
- Usa **modelos de linguagem** (ex: MiniLM, BGE) para gerar embeddings densos
- Cada documento/query → vetor contínuo de 384d (MiniLM) ou 1024d (BGE-Large)
- Similaridade captura **significado semântico**, não apenas palavras exatas

## Modelos comparados no paper
| Modelo | Dimensão | Parâmetros | Uso |
|--------|----------|------------|-----|
| **MiniLM-L6-v2** | 384 | 22M | Rápido, eficiente |
| **BGE-Large** | 1024 | 335M | Maior qualidade |

## Por que MiniLM pode superar BGE no híbrido?
O paper mostra que embeddings menores podem ter **melhor alinhamento com LLMs** durante o reranking (fenômeno chamado "FAISS Hybrid Paradox").

In [5]:
from src.retrievers.dense_faiss import DenseFaiss

# Escolha do modelo (altere para testar)
dense_model = "sentence-transformers/all-MiniLM-L6-v2"  
# Alternativa: "BAAI/bge-large-en-v1.5"

dense_r = DenseFaiss(
    model_name=dense_model,
    device="cuda:0",              # Use "cuda:0" se tiver GPU disponível
    query_prefix="",
    doc_prefix="",
    use_faiss=True,
    artifact_dir=None,
    index_name="dense.index",
)

print(f"🔧 Construindo índice denso com {dense_model}...")
print(f"   Dimensão: {dense_r.dim}d")
dense_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
dense_results = dense_r.retrieve(queries, k=3)

for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = dense_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        doc = next(d for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {doc.title}")

/teamspace/studios/this_studio/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Construindo índice denso com sentence-transformers/all-MiniLM-L6-v2...
   Dimensão: 384d
2025-10-30 00:38:59 | INFO     | retriever.dense | [dense_faiss.py:75] | 🚀 Building Dense Index (24 documentos)
2025-10-30 00:38:59 | INFO     | retriever.dense | [logging.py:199] | ⏱️  Encoding documents - iniciando...
2025-10-30 00:38:59 | INFO     | retriever.dense | [logging.py:220] | ✓ Encoding documents - concluído em 7.0ms
2025-10-30 00:38:59 | INFO     | retriever.dense | [logging.py:199] | ⏱️  Construindo FAISS IndexFlatIP - iniciando...
2025-10-30 00:38:59 | INFO     | retriever.dense | [logging.py:220] | ✓ Construindo FAISS IndexFlatIP - concluído em 0.6ms
2025-10-30 00:38:59 | INFO     | retriever.dense | [dense_faiss.py:92] |   ✓ FAISS IndexFlatIP: 24 vetores, dim=384

🔍 Executando retrieval (top-3 por query)...

📊 Query [Q1]: Are mRNA vaccines effective against COVID-19?
  1. B1 (score=0.7346) - COVID-19 mRNA vaccines
  2. B2 (score=0.4246) - mRNA vaccine safety profile
  3. X2 (sco

### 📈 Análise dos Resultados Dense

**Observe**:
- Melhor captura de **sinônimos** e **paráfrases** (ex: "effective" ≈ "efficacy")
- Scores geralmente mais **uniformes** do que TF-IDF (espaço denso é mais suave)
- Pode ranquear documentos **tematicamente relacionados** mesmo sem overlap lexical

**Experimento**: Troque para `"BAAI/bge-large-en-v1.5"` e compare os resultados!

---
# 🕸️ Parte 3: Graph Retriever (Entidades + Embeddings)

## Visão Geral
O retriever de grafo implementa o **terceiro modo (g)** do paper:
1. **Extração de Entidades**: Identifica "nomes" importantes no texto (pessoas, locais, termos técnicos)
2. **IDF de Entidades**: Calcula importância relativa de cada entidade no corpus
3. **Embedding de Entidades**: Cada entidade → vetor denso (ex: BGE-Large 1024d)
4. **Agregação TF-IDF**: Vetor do documento = Σ [TF(e) × IDF(e) × emb(e)]
5. **Normalização L2**: Vetor final é normalizado para comparação por cosseno

## Fórmula

g(text) = L2_norm( Σ_{e ∈ entities(text)} TF(e, text) × IDF(e) × embedding(e) )


## Por que é útil?
- Captura **sinal estruturado** de entidades nomeadas
- Útil em domínios com **terminologia específica** (medicina, finanças, produtos)
- Complementa sinais denso e lexical no híbrido

## 🔍 Passo 1: Extração de Entidades (NER unificado + regras leves)

Agora usamos um único backbone NER para todos os domínios: **spaCy en_core_web_trf** (RoBERTa-base).
Para “vitaminar” a cobertura, adicionamos um EntityRuler com padrões leves:

- Financeiro: ORG/MONEY_TERM básicos (ex.: “Federal Reserve”, “interest rates”) e um padrão simples de TICKER (regex).
- Biomédico: termos científicos comuns (ex.: “COVID-19”, “mRNA”, “clinical trials”).
- Além disso, usamos noun chunks (sintagmas nominais), que ajudam em termos compostos.

Isso busca equilíbrio entre SciFact (ciência/biomédico), NFCorpus (saúde/nutrição) e FIQA (finanças) sem manter três NERs diferentes.

In [6]:
# Extração de entidades com spaCy en_core_web_trf + EntityRuler (regras leves)
# Se o modelo não estiver instalado, você pode fazer:
#   !python -m spacy download en_core_web_trf
# Caso indisponível, cairemos para "en_core_web_sm".

doc_text = docs[0].title + " " + docs[0].text
print(f"📄 Documento D1:\n{doc_text}\n")

try:
    import spacy
    try:
        # mantém parser ativo para noun_chunks
        nlp = spacy.load("en_core_web_trf", disable=["tagger", "lemmatizer"])
        print("✓ Usando spaCy (transformer): en_core_web_trf")
    except Exception:
        nlp = spacy.load("en_core_web_sm", disable=["tagger", "lemmatizer"])
        print("✓ Usando spaCy: en_core_web_sm")

    # Adiciona EntityRuler com padrões leves (finance/biomed) + um padrão simples de TICKER (regex)
    before = "ner" if "ner" in nlp.pipe_names else None
    ruler = nlp.add_pipe("entity_ruler", before=before)
    patterns = [
        # Finance
        {"label": "ORG", "pattern": "Federal Reserve"},
        {"label": "ORG", "pattern": "Fed"},
        {"label": "ORG", "pattern": "Nasdaq"},
        {"label": "ORG", "pattern": "Dow Jones"},
        {"label": "ORG", "pattern": "S&P 500"},
        {"label": "MONEY_TERM", "pattern": "interest rates"},
        {"label": "MONEY_TERM", "pattern": "equity market"},
        # TICKER (regex super simples just for demo)
        {"label": "TICKER", "pattern": [{"TEXT": {"REGEX": "^[A-Z]{1,5}$"}}]},
        # Biomed
        {"label": "DISEASE_TERM", "pattern": "COVID-19"},
        {"label": "DISEASE_TERM", "pattern": "SARS-CoV-2"},
        {"label": "BIOMED_TERM", "pattern": "mRNA"},
        {"label": "BIOMED_TERM", "pattern": "clinical trials"},
    ]
    ruler.add_patterns(patterns)

    doc = nlp(doc_text)

    # Entidades nomeadas (NER + regras)
    entities = []
    print("\n🏷️  Entidades Nomeadas Detectadas:")
    for ent in doc.ents:
        entities.append(ent.text.lower().strip())
        print(f"   - '{ent.text}' [{ent.label_}]")

    # Noun chunks
    print("\n📦 Noun Chunks Detectados:")
    for chunk in doc.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text not in entities:
            entities.append(chunk_text)
        print(f"   - '{chunk.text}'")

    print(f"\n✓ Total de entidades únicas: {len(set(entities))}")
    print(f"  Entidades: {sorted(set(entities))}")

except Exception as e:
    print(f"⚠️  spaCy não disponível: {e}")
    print("   Usando fallback regex (detecta termos capitalizados e longos)")

    import re
    pattern = re.compile(r"[A-Za-z][A-Za-z0-9_\-/\.]{1,}")
    candidates = []
    for tok in pattern.findall(doc_text):
        if len(tok) >= 10 or tok[0].isupper():
            candidates.append(tok.lower())
    print(f"\n  Entidades detectadas (fallback): {sorted(set(candidates))}")

📄 Documento D1:
COVID-19 mRNA vaccines Clinical trials show efficacy of mRNA vaccines against COVID-19.



✓ Usando spaCy (transformer): en_core_web_trf

🏷️  Entidades Nomeadas Detectadas:
   - 'COVID-19' [DISEASE_TERM]
   - 'mRNA' [BIOMED_TERM]
   - 'mRNA' [BIOMED_TERM]
   - 'COVID-19' [DISEASE_TERM]

📦 Noun Chunks Detectados:

✓ Total de entidades únicas: 2
  Entidades: ['covid-19', 'mrna']


### 💡 Interpretação da Extração (NER unificado + regras)

- Backbone: `en_core_web_trf` (RoBERTa-base) com bom recall geral (ORG, PERSON, MONEY, DATE, GPE...).
- Regras leves (EntityRuler):
  - Finance: “Federal Reserve”, “interest rates”, “equity market”, tickers simples.
  - Biomed: “COVID-19”, “SARS-CoV-2”, “mRNA”, “clinical trials”.
- Noun Chunks: sintagmas nominais como “mRNA vaccines”, “interest rates”.
- Normalização: lowercase, remoção de pontuação, mínimo de 2 caracteres.

Vantagem: um único NER robusto a múltiplos domínios com complementos mínimos por regra.

## 🧮 Passo 2: Cálculo do IDF de Entidades

Após extrair entidades de **todos** os documentos, calculamos o IDF:

IDF(e) = log((1 + N) / (1 + DF(e))) + 1.0


Onde:
- **N** = número total de documentos
- **DF(e)** = número de documentos que contêm a entidade `e`

**Intuição**: Entidades raras (baixo DF) recebem maior peso; entidades comuns (alto DF) são penalizadas.

In [7]:
# Cálculo manual de DF/IDF com min_df=2 (opcional, para inspecionar)
from collections import defaultdict
import re
import numpy as np

print("🔢 Calculando IDF de Entidades no Corpus (min_df=2)\n")

def extract_entities_simple(text):
    pattern = re.compile(r"[A-Za-z][A-Za-z0-9_\-/\.]{1,}")
    entities = []
    for tok in pattern.findall(text):
        if len(tok) >= 10 or tok[0].isupper():
            entities.append(tok.lower().strip(".,;:()[]{}"))
    return [e for e in entities if len(e) >= 2]

df = defaultdict(int)
N = len(docs)

for d in docs:
    ents = set(extract_entities_simple((d.title or "") + " " + (d.text or "")))
    for e in ents:
        df[e] += 1

# aplica min_df=2
df2 = {e:c for e,c in df.items() if c >= 2}

entity_idf = {e: (np.log((1+N)/(1+c)) + 1.0) for e,c in df2.items()}
sorted_entities = sorted(entity_idf.items(), key=lambda x: x[1], reverse=True)

print("📊 Top 15 Entidades por IDF (DF ≥ 2):\n")
for e, idf in sorted_entities[:15]:
    print(f"   {e:<22} | DF={df[e]:>2} | IDF={idf:.4f}")
print(f"\n✓ Entidades (DF ≥ 2): {len(entity_idf)} de {len(df)} únicas")

🔢 Calculando IDF de Entidades no Corpus (min_df=2)

📊 Top 15 Entidades por IDF (DF ≥ 2):

   mediterranean          | DF= 2 | IDF=3.1203
   valuations             | DF= 2 | IDF=3.1203
   us                     | DF= 2 | IDF=3.1203
   nasdaq                 | DF= 2 | IDF=3.1203
   ai                     | DF= 2 | IDF=3.1203
   clinical               | DF= 3 | IDF=2.8326
   cardiovascular         | DF= 3 | IDF=2.8326
   covid-19               | DF= 4 | IDF=2.6094

✓ Entidades (DF ≥ 2): 8 de 67 únicas


### 📈 Interpretação do IDF

**Entidades com IDF alto** (raras):
- Aparecem em poucos documentos
- São mais **discriminativas** (ajudam a diferenciar documentos)

**Entidades com IDF baixo** (comuns):
- Aparecem em muitos documentos
- São menos úteis para distinguir conteúdo
- Exemplo: termos genéricos que aparecem em múltiplos contextos

**Por que isso importa?** 
O IDF pondera a contribuição de cada entidade no vetor final, dando mais peso a termos discriminativos.

## 🎯 Passo 3: Embedding de Entidades

Cada entidade é transformada em um **vetor denso** usando um modelo HF (ex: BGE-Large).

**Exemplo**: 
- Entidade: `"mRNA vaccines"`
- Embedding: vetor de 1024 dimensões (se BGE-Large)

**Por que embeddings de entidades?**
- Captura **similaridade semântica entre entidades** (ex: "mRNA" ≈ "RNA-based")
- Permite que entidades relacionadas contribuam de forma similar ao vetor do documento

In [8]:
# Vamos ver como uma entidade é transformada em embedding

from src.encoders.encoders import HFSemanticEncoder, l2norm

print("🧬 Gerando Embeddings de Entidades\n")

# Carrega encoder (mesmo usado no retriever de grafo)
entity_encoder = HFSemanticEncoder(
    model_name="BAAI/bge-large-en-v1.5",  # BGE-Large (1024d)
    device=None,
)

print(f"✓ Modelo: {entity_encoder.model_name}")
print(f"✓ Dimensão: {entity_encoder.dim}d\n")

# Embute algumas entidades exemplo
sample_entities = ["mrna vaccines", "covid-19", "equity market", "gnns"]

print("📊 Embeddings de Entidades (primeiras 5 dimensões):\n")
embeddings = {}
for ent in sample_entities:
    emb = l2norm(entity_encoder.encode_text(ent, is_query=False))
    embeddings[ent] = emb
    norm = np.linalg.norm(emb)
    print(f"   {ent:<20} → [{emb[:5]}...] || norm={norm:.4f}")

print("\n✓ Embeddings gerados e normalizados (L2 norm ≈ 1.0)")

🧬 Gerando Embeddings de Entidades



✓ Modelo: BAAI/bge-large-en-v1.5
✓ Dimensão: 1024d

📊 Embeddings de Entidades (primeiras 5 dimensões):

   mrna vaccines        → [[0.0429553  0.00623018 0.02103916 0.0337704  0.00673586]...] || norm=1.0000
   covid-19             → [[-0.03450392 -0.00453843 -0.00358562  0.06227142  0.00305257]...] || norm=1.0000
   equity market        → [[-0.01458916  0.04924496 -0.02150347  0.01151251  0.00439941]...] || norm=1.0000
   gnns                 → [[-0.00744154 -0.00087572 -0.00218375 -0.01080718  0.0085467 ]...] || norm=1.0000

✓ Embeddings gerados e normalizados (L2 norm ≈ 1.0)


### 🔗 Similaridade entre Entidades

Vamos calcular a **similaridade de cosseno** entre pares de entidades para verificar se o modelo captura relações semânticas.

In [9]:
# Calcula similaridade de cosseno entre pares de entidades

def cosine_sim(v1, v2):
    return float(np.dot(v1, v2))  # Já normalizados (L2), então dot = cosseno

print("🔗 Similaridade entre Entidades (cosseno):\n")

pairs = [
    ("mrna vaccines", "covid-19"),      # relacionadas (domínio médico)
    ("equity market", "gnns"),          # não relacionadas (finanças vs ML)
    ("mrna vaccines", "equity market"), # não relacionadas (medicina vs finanças)
]

for e1, e2 in pairs:
    if e1 in embeddings and e2 in embeddings:
        sim = cosine_sim(embeddings[e1], embeddings[e2])
        print(f"   '{e1}' ↔ '{e2}': {sim:.4f}")

print("\n💡 Entidades semanticamente relacionadas têm similaridade maior!")

🔗 Similaridade entre Entidades (cosseno):

   'mrna vaccines' ↔ 'covid-19': 0.5725
   'equity market' ↔ 'gnns': 0.4921
   'mrna vaccines' ↔ 'equity market': 0.5086

💡 Entidades semanticamente relacionadas têm similaridade maior!


## 🧩 Passo 4: Agregação TF-IDF de Entidades

Agora combinamos tudo para gerar o **vetor do documento**:

g(doc) = L2_norm( Σ_{e ∈ doc} TF(e, doc) × IDF(e) × embedding(e) )


Onde:
- **TF(e, doc)** = frequência da entidade no documento
- **IDF(e)** = inverse document frequency (calculado no passo 2)
- **embedding(e)** = vetor denso da entidade (calculado no passo 3)

**Algoritmo**:
1. Extrair entidades do documento
2. Contar TF de cada entidade
3. Para cada entidade: peso = TF × IDF
4. Somar: vetor_final = Σ (peso × embedding)
5. Normalizar L2

In [10]:
# Vamos calcular manualmente o vetor de grafo para o documento D1

doc_d1 = docs[0]
text_d1 = (doc_d1.title or "") + " " + (doc_d1.text or "")
print(f"📄 Documento D1:\n{text_d1}\n")

# 1. Extrai entidades
entities_d1 = extract_entities_simple(text_d1)
print(f"🏷️  Entidades extraídas: {entities_d1}\n")

# 2. Conta TF
from collections import Counter
tf_d1 = Counter(entities_d1)
print(f"🔢 Term Frequency (TF):")
for e, count in tf_d1.items():
    print(f"   {e}: {count}")

# 3. Calcula vetor agregado (manual)
print(f"\n🧮 Agregação TF-IDF × Embeddings:")
vec_d1 = np.zeros(entity_encoder.dim, dtype=np.float32)

for e, tf in tf_d1.items():
    if e in entity_idf:  # entidade conhecida no corpus
        idf = entity_idf[e]
        weight = tf * idf
        
        # Gera embedding da entidade (ou usa cache)
        if e not in embeddings:
            embeddings[e] = l2norm(entity_encoder.encode_text(e, is_query=False))
        emb = embeddings[e]
        
        vec_d1 += weight * emb
        print(f"   {e:<20} | TF={tf} | IDF={idf:.4f} | weight={weight:.4f}")

# 4. Normaliza L2
vec_d1 = l2norm(vec_d1)
print(f"\n✓ Vetor final (primeiras 5 dims): {vec_d1[:5]}")
print(f"✓ Norma L2: {np.linalg.norm(vec_d1):.4f}")

📄 Documento D1:
COVID-19 mRNA vaccines Clinical trials show efficacy of mRNA vaccines against COVID-19.

🏷️  Entidades extraídas: ['covid-19', 'clinical', 'covid-19']

🔢 Term Frequency (TF):
   covid-19: 2
   clinical: 1

🧮 Agregação TF-IDF × Embeddings:
   covid-19             | TF=2 | IDF=2.6094 | weight=5.2189


   clinical             | TF=1 | IDF=2.8326 | weight=2.8326

✓ Vetor final (primeiras 5 dims): [-0.03098487 -0.00815338 -0.00128427  0.05437236 -0.01989531]
✓ Norma L2: 1.0000


### 💡 Interpretação da Agregação

**Contribuição de cada entidade**:
- Entidades **frequentes** no documento (alto TF) contribuem mais
- Entidades **raras** no corpus (alto IDF) têm maior peso relativo
- Embedding captura o **significado semântico** da entidade

**Resultado final**:
- Vetor denso de 1024d (se BGE-Large) normalizado
- Representa o documento como uma **composição ponderada de suas entidades**
- Permite comparação por cosseno com queries que também passam pelo mesmo processo

## 🚀 Retriever de Grafo Completo (com NER unificado)

Aqui usamos o `GraphRetriever` com backend NER = spaCy `en_core_web_trf`.
Observação:
- O código de produção (classe `EntityEncoderReal`) já suporta EntityRuler e regras leves.
- Em produção, essas regras podem ser ligadas via flags no `NERConfig` (no pipeline interno).
- No laboratório acima, demonstramos como “vitaminar” manualmente via spaCy para visualização.

O pipeline automatiza:
1) Fit (extração + IDF)
2) Build (vectorização e index FAISS)
3) Retrieve (query → vetor de grafo → busca por cosseno)

In [11]:
from src.retrievers.graph_faiss import GraphRetriever

graph_r = GraphRetriever(
    graph_model_name="BAAI/bge-large-en-v1.5",
    device=None,                  # "cuda:0" se houver GPU
    ner_backend="spacy",
    ner_model="en_core_web_trf",  # backbone único
    ner_use_noun_chunks=True,
    ner_batch_size=64,
    ner_n_process=1,
    ner_allowed_labels=None,      # manter amplo para o lab
    min_df=1,                     # entidades precisam aparecer em >=2 docs
    use_faiss=True,
    artifact_dir=None,
    index_name="graph.index",
    entity_artifact_dir=None,
    entity_force_rebuild=False,
)

print("🔧 Construindo índice de grafo...")
print(f"   Modelo: {graph_r.vec.encoder.model_name}")
print(f"   Dimensão: {graph_r.vec.dim}d")
graph_r.build_index(docs)

print("\n🔍 Executando retrieval (top-3 por query)...")
graph_results = graph_r.retrieve(queries, k=3)

for q in queries:
    print(f"\n📊 Query [{q.query_id}]: {q.text}")
    pairs = graph_results.get(q.query_id, [])
    for rank, (doc_id, score) in enumerate(pairs, 1):
        title = next(d.title for d in docs if d.doc_id == doc_id)
        print(f"  {rank}. {doc_id} (score={score:.4f}) - {title}")

🔧 Construindo índice de grafo...
   Modelo: BAAI/bge-large-en-v1.5
   Dimensão: 1024d
2025-10-30 00:39:18 | INFO     | retriever.graph | [graph_faiss.py:91] | 🚀 Building Graph Index (24 documentos)
2025-10-30 00:39:18 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Fit GraphVectorizer no corpus - iniciando...
2025-10-30 00:39:18 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-10-30 00:39:19 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 958.6ms
2025-10-30 00:39:19 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=1024, ents=5
2025-10-30 00:39:19 | INFO     | retriever.graph | [logging.py:220] | ✓ Fit GraphVectorizer no corpus - concluído em 961.3ms
2025-10-30 00:39:19 | INFO     | retriever.graph | [logging.py:199] | ⏱️  Encoding documents (Graph) - iniciando...
2025-10-30 00:39:30 | INFO     | retriever.graph | [logging.py:220] | ✓ Encoding documents (Graph) -

### 📈 Análise dos Resultados Graph

**Observe**:
- Queries com **entidades nomeadas fortes** (ex: "mRNA", "COVID", "entity embeddings") devem ranquear bem documentos relacionados
- O retriever de grafo é especialmente forte quando:
  - Documento e query compartilham **entidades raras** (alto IDF)
  - Entidades têm **alta similaridade semântica** nos embeddings
- Pode capturar relações que TF-IDF perde (paráfrases de entidades) e que Dense não enfatiza (importância de termos específicos)

---
# 📊 Comparação Lado a Lado dos Três Retrievers

Vamos consolidar os resultados dos três métodos para facilitar a análise comparativa.

In [12]:
import pandas as pd

def format_results(results, name):
    """Formata resultados em DataFrame para visualização"""
    rows = []
    for q in queries:
        pairs = results.get(q.query_id, [])[:3]
        for rank, (doc_id, score) in enumerate(pairs, 1):
            doc = next(d for d in docs if d.doc_id == doc_id)
            rows.append({
                'Query': q.query_id,
                'Retriever': name,
                'Rank': rank,
                'Doc': doc_id,
                'Score': f"{score:.4f}",
                'Title': doc.title[:40]
            })
    return pd.DataFrame(rows)

# Concatena resultados
df_tfidf = format_results(tfidf_results, "TF-IDF")
df_dense = format_results(dense_results, "Dense")
df_graph = format_results(graph_results, "Graph")

df_all = pd.concat([df_tfidf, df_dense, df_graph], ignore_index=True)

print("📊 Comparação dos Retrievers (Top-3 por Query):\n")
for qid in ["Q1", "Q2", "Q3"]:
    print(f"\n{'='*80}")
    print(f"Query {qid}: {next(q.text for q in queries if q.query_id == qid)}")
    print('='*80)
    subset = df_all[df_all['Query'] == qid]
    print(subset[['Retriever', 'Rank', 'Doc', 'Score', 'Title']].to_string(index=False))

📊 Comparação dos Retrievers (Top-3 por Query):


Query Q1: Are mRNA vaccines effective against COVID-19?
Retriever  Rank Doc  Score                       Title
   TF-IDF     1  B1 0.9198      COVID-19 mRNA vaccines
   TF-IDF     2  B2 0.7802 mRNA vaccine safety profile
   TF-IDF     3  B5 0.5067          Long COVID studies
    Dense     1  B1 0.7346      COVID-19 mRNA vaccines
    Dense     2  B2 0.4246 mRNA vaccine safety profile
    Dense     3  X2 0.3059     COVID-19 and US economy
    Graph     1  B3 0.0000         SARS-CoV-2 variants
    Graph     2  B2 0.0000 mRNA vaccine safety profile
    Graph     3  B1 0.0000      COVID-19 mRNA vaccines

Query Q2: How do interest rates impact equity volatility and valuations?
Retriever  Rank Doc  Score                                Title
   TF-IDF     1  F1 0.8616 Interest rates and equity volatility
   TF-IDF     2  F2 0.4282                   US economy outlook
   TF-IDF     3  F5 0.2362              Dow Jones and dividends
    Dense     1

### 🔍 Análise Comparativa

| Retriever | Força | Fraqueza |
|-----------|-------|----------|
| **TF-IDF** | ✓ Match exato de termos<br>✓ Rápido e interpretável | ✗ Não captura sinônimos<br>✗ Sensível a vocabulário |
| **Dense** | ✓ Similaridade semântica<br>✓ Generalização | ✗ Pode perder signal lexical específico<br>✗ Menos interpretável |
| **Graph** | ✓ Captura entidades importantes<br>✓ Ponderação por IDF | ✗ Depende da qualidade do NER<br>✗ Mais lento (extração + embedding) |

**Conclusão**: Cada retriever captura um **sinal complementar**. O retriever **híbrido** combina os três para obter o melhor de todos.

---
# 🔬 Exploração Adicional: Dimensões e Normalização

Vamos verificar as dimensões e normas L2 dos vetores gerados por cada retriever para confirmar que estão corretamente normalizados.

In [15]:
# Verifica dimensões e normalização

print("📐 Dimensões e Normalização dos Vetores\n")

# TF-IDF
from src.vectorizers.tfidf_vectorizer import TFIDFVectorizer
tf_vec = TFIDFVectorizer(dim=1000, min_df=1, backend="sklearn")
tf_vec.fit_corpus([(d.title or "") + " " + (d.text or "") for d in docs])
v_tf = tf_vec.encode_text(queries[0].text)
print(f"TF-IDF:")
print(f"   Dim: {v_tf.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_tf):.6f} | Sparsity: {np.sum(v_tf == 0) / len(v_tf) * 100:.1f}%")

# Dense (MiniLM)
from src.vectorizers.dense_vectorizer import DenseVectorizer
dv = DenseVectorizer(model_name="sentence-transformers/all-MiniLM-L6-v2", device=None)
v_d = dv.encode_query(queries[0].text)
print(f"\nDense (MiniLM):")
print(f"   Dim: {v_d.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_d):.6f} | Sparsity: {np.sum(v_d == 0) / len(v_d) * 100:.1f}%")

# Graph
from src.vectorizers.graph_vectorizer import GraphVectorizer
gv = GraphVectorizer(graph_model_name="BAAI/bge-large-en-v1.5", device=None,
                     ner_backend="spacy", ner_model="en_core_web_trf",
                     ner_use_noun_chunks=True, min_df=1)
gv.fit_corpus([(d.title or "") + " " + (d.text or "") for d in docs])

qtxt = queries[0].text
ents_q = gv.encoder._extract_entities_batch([qtxt])[0]
ents_in_vocab = [e for e in ents_q if e in gv.encoder.ent2idf]
print("Entidades query:", ents_q)
print("No vocabulário (IDF):", ents_in_vocab)
v_g = gv.encode_text(qtxt)
print("||v_g||:", float(np.linalg.norm(v_g)))

print(f"\nGraph (BGE-Large):")
print(f"   Dim: {v_g.shape[0]:>4}d | L2 norm: {np.linalg.norm(v_g):.6f} | Sparsity: {np.sum(v_g == 0) / len(v_g) * 100:.1f}%")

print("\n✓ Todos os vetores estão normalizados (L2 norm ≈ 1.0)")
print("✓ TF-IDF é esparso; Dense e Graph são densos")

📐 Dimensões e Normalização dos Vetores

2025-10-30 00:44:53 | INFO     | tfidf.vectorizer | [logging.py:199] | ⏱️  Fit TF-IDF - iniciando...
2025-10-30 00:44:53 | INFO     | tfidf.vectorizer | [logging.py:220] | ✓ Fit TF-IDF - concluído em 2.3ms
2025-10-30 00:44:53 | INFO     | tfidf.vectorizer | [tfidf_vectorizer.py:19] | ✓ TF-IDF fitted: vocab_size=171
TF-IDF:
   Dim:  171d | L2 norm: 1.000000 | Sparsity: 97.1%

Dense (MiniLM):
   Dim:  384d | L2 norm: 1.000000 | Sparsity: 0.0%
2025-10-30 00:44:56 | INFO     | graph.vectorizer | [logging.py:199] | ⏱️  Fit Graph (NER + IDF) - iniciando...
2025-10-30 00:44:57 | INFO     | graph.vectorizer | [logging.py:220] | ✓ Fit Graph (NER + IDF) - concluído em 1.28s
2025-10-30 00:44:57 | INFO     | graph.vectorizer | [graph_vectorizer.py:49] | ✓ Graph fitted: dim=1024, ents=5
Entidades query: []
No vocabulário (IDF): []
||v_g||: 0.0

Graph (BGE-Large):
   Dim: 1024d | L2 norm: 0.000000 | Sparsity: 100.0%

✓ Todos os vetores estão normalizados (L2 n

### 📊 Interpretação das Dimensões

**TF-IDF** (1000d):
- Dimensão = tamanho do vocabulário (limitado por `max_features`)
- **Altamente esparso**: maioria das dimensões é zero (termos não presentes)
- Sparsity típica: 95-99%

**Dense MiniLM** (384d):
- Dimensão fixa definida pelo modelo
- **Totalmente denso**: todas as dimensões têm valores não-zero
- Menor que BGE, mas ainda captura semântica eficazmente

**Graph BGE-Large** (1024d):
- Dimensão = dimensão do encoder de entidades
- **Denso**: composição ponderada de embeddings de entidades
- Maior dimensão → maior capacidade representacional

**Normalização L2**: Todos os vetores têm norma ≈ 1.0, permitindo comparação direta por inner-product (equivalente a cosseno).